In [ ]:
import cPickle
import pandas as pd
import numpy as np
import seaborn as sn
from matplotlib import pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

### Pre-process data

In [ ]:
def unpickle(file):
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

def reshape_img(img):
    img = img.reshape((3,32*32))
    v = np.zeros((32,32,3),dtype=np.uint8)
    for i in range(32):
        for j in range(32):
            v[i][j][0] = img[0][i*32+j]
            v[i][j][1] = img[1][i*32+j]
            v[i][j][2] = img[2][i*32+j]  
    return v


#### Train set

#### Test set

### Load Dataset

In [ ]:
DATASET_DIR = "dataset/"
DATASET_FILE = "train.pkl"

LABELS = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
# LABELS = ['airplane','automobile','bird','cat','deer','frog','horse','ship','truck']

dataset = pd.read_pickle(DATASET_DIR + DATASET_FILE)
dataset = dataset.sample(3000,random_state=7)
# dataset = dataset[dataset.label != 3]
# dataset = dataset[dataset.label != 5]

SUFFIX = 35

#### Train-Test split

In [ ]:
dataset_train, dataset_test = train_test_split(dataset,test_size=.2)

## CNN

In [ ]:
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Convolution2D,MaxPooling2D,ZeroPadding2D
from keras.layers.convolutional import Conv2D
from keras.constraints import maxnorm
from keras.optimizers import SGD

In [ ]:
X_train = []
for v in dataset_train.img.values:
    X_train.append(np.array(v))
X_train = np.array(X_train)

X_test = []
for v in dataset_test.img.values:
    X_test.append(np.array(v))
X_test = np.array(X_test)

y_train = to_categorical(dataset_train.label.values)
y_test = to_categorical(dataset_test.label.values)

#### Compile Model

In [ ]:
EPOCHS = 5
L_RATE = 0.01
DECAY = L_RATE/EPOCHS

sgd = SGD(lr=L_RATE, momentum=0.9, decay=DECAY, nesterov=False)

# Create Keras model
model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='valid', activation='relu',input_shape=(32,32,3)))
model.add(Dropout(0.2))

model.add(Convolution2D(32, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu'))
model.add(Dropout(0.2))

model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu'))
model.add(ZeroPadding2D((1, 1)))

model.add(Convolution2D(128, 3, 3, border_mode='valid', activation='relu'))
model.add(Dropout(0.2))

model.add(Convolution2D(128, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(ZeroPadding2D((1, 1)))

model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# model.summary()

#### Fit

In [ ]:
model.fit(X_train, y_train,validation_split=0.2,batch_size=64, nb_epoch=EPOCHS,verbose=1)

#### Evaluate

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)
score

#### Predict

In [ ]:
preds = []
for pred in model.predict(X_test):
    preds.append(pred.argmax())
    
y_test_labels = []
for y in y_test:
    y_test_labels.append(y.argmax())

In [ ]:
f = open("output/score"+str(SUFFIX)+".txt",'w+')
f.write("Dataset size: %d\n"%(len(dataset)))
f.write("Epochs: %d\n"%(EPOCHS))
f.write("%s\n\n" % str(score))
f.write(classification_report(y_test_labels,preds,target_names=LABELS))
f.close()

In [ ]:
cm = confusion_matrix(preds,y_test_labels)
df_cm = pd.DataFrame(cm, index = LABELS,
                  columns = LABELS)
sn.heatmap(df_cm,annot=True,fmt="")
plt.savefig('output/fig'+str(SUFFIX)+'.png')
plt.show()

In [ ]:
out = pd.DataFrame({'Predicted': preds,'Actual': y_test_labels})
out.to_csv("output/preds_"+str(SUFFIX)+".csv")

In [ ]:
print classification_report(y_test_labels,preds,target_names=LABELS)